In [ ]:
include("main.jl")
using PyCall
using CairoMakie
sepsis_gym = pyimport("custom_sepsis")
using Statistics


In [ ]:
struct MeansType
    individual_runs::Vector{Vector{Float64}}
    mean_rewards::Vector{Float64}
    smoothed_mean::Vector{Float64}
    smoothed_std::Vector{Float64}
    keys_of_smoothed::Vector{Float64}
end
function calculate_mean_rewards(results, window_size)
    rews = Dict()
    means = Dict()
    for (type, run) in results
        all_keys = [sort(collect(keys(model.mean_rewards))) for model in run]
        min_keys = minimum(length.(all_keys))
        ks = all_keys[1][1:min_keys]
        filled_rewards = []
        all_rewards = []
        rews = []
        for model in run
            rewards = [model.mean_rewards[1]]
            for i in 2:min_keys
                key = all_keys[1][i]
                mult_factor = key - all_keys[1][i-1]
                push!(rewards, fill(model.mean_rewards[key], mult_factor)...)
            end
            push!(all_rewards, [model.mean_rewards[k] for k in ks])
            push!(filled_rewards, rewards)
            push!(rews, rewards)
        end
        mean_rewards = mean(all_rewards)
        std_rewards = std(all_rewards)
        means[type] = MeansType( rews,  mean(filled_rewards),  moving_avg(mean_rewards, window_size),  Float64.(moving_avg(std_rewards, window_size)),  Float64.(ks))
    end
    return means
end

In [ ]:
ts = Dict(
    :Simple100 => [sepsis_gym.DirThompsonSampling.load_json("json/dirichlet/ts/Simple-$i.json") for i in 0:2],
    :Medium100 => [sepsis_gym.DirThompsonSampling.load_json("json/dirichlet/ts/Medium-$i.json") for i in 0:2],
    :None100 => [sepsis_gym.DirThompsonSampling.load_json("json/dirichlet/ts/None-$i.json") for i in 0:2],
    :Softmax100 => [load_jld("data/mcmc/runs/SoftmaxPPL-$i.jld")  for i in 1:3],
    :SimplePPL100 => [load_jld("data/mcmc/runs/SimplePPL-$i.jld")  for i in 1:3],
    :Simple1 => [sepsis_gym.DirThompsonSampling.load_json("json/dirichlet/ts/Simple-every-$i.json") for i in 1:3],
    :Medium1 => [sepsis_gym.DirThompsonSampling.load_json("json/dirichlet/ts/Medium-every-$i.json") for i in 1:3],
    :Softmax1 => [load_jld("data/mcmc/runs/SoftmaxPPL-every-$i.jld")  for i in 1:3],
    :SimplePPL1 => [load_jld("data/mcmc/runs/SimplePPL-every-$i.jld")  for i in 1:3],
)

In [ ]:
means = calculate_mean_rewards(ts, 5)

In [ ]:
using JSON3

function save_means_to_json(means, directory)
    for (key, value) in means
        file_path = joinpath(directory, "$(key).json")
        json_data = Dict(
            "individual_runs" => value.individual_runs,
            "mean_rewards" => value.mean_rewards,
            "smoothed_mean" => value.smoothed_mean,
            "smoothed_std" => value.smoothed_std,
            "keys_of_smoothed" => value.keys_of_smoothed,
            "name"=> key,
            "info" => Dict(),
        )
        JSON3.write(file_path, json_data)
    end
end

# Save means to JSON files in the "means_json" directory
save_means_to_json(means, "json/ts/")